In [161]:
import os
import numpy as np
import pandas as pd
import re

from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [162]:
from sklearn.naive_bayes import MultinomialNB

In [163]:
data_test = pd.read_csv("donnees/Corona_NLP_test.csv",encoding='latin-1')
data_train = pd.read_csv("donnees/Corona_NLP_train.csv",encoding='latin-1')

print(data_train.shape)
print(data_test.shape)

data_train

(41157, 6)
(3798, 6)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [164]:
#Replace Extremely
data_test["Sentiment"] = data_test["Sentiment"].replace("Extremely Negative","Negative")
data_test["Sentiment"] = data_test["Sentiment"].replace("Extremely Positive","Positive")

data_train["Sentiment"] = data_train["Sentiment"].replace("Extremely Negative","Negative")
data_train["Sentiment"] = data_train["Sentiment"].replace("Extremely Positive","Positive")


#Remove Urls and HTML links
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

data_train["OriginalTweet"]=data_train["OriginalTweet"].apply(lambda x:remove_urls(x))
data_test["OriginalTweet"]=data_test["OriginalTweet"].apply(lambda x:remove_urls(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

data_train["OriginalTweet"]=data_train["OriginalTweet"].apply(lambda x:remove_html(x))
data_test["OriginalTweet"]=data_test["OriginalTweet"].apply(lambda x:remove_html(x))

# Lower casing
def lower(text):
    low_text= text.lower()
    return low_text

data_train["OriginalTweet"]=data_train["OriginalTweet"].apply(lambda x:lower(x))
data_test["OriginalTweet"]=data_test["OriginalTweet"].apply(lambda x:lower(x))


# Number removal
def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove

data_train["OriginalTweet"]=data_train["OriginalTweet"].apply(lambda x:remove_num(x))
data_test["OriginalTweet"]=data_test["OriginalTweet"].apply(lambda x:remove_num(x))

#Remove stopwords & Punctuations
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data_train["OriginalTweet"]=data_train["OriginalTweet"].apply(lambda x:remove_stopwords(x))
data_test["OriginalTweet"]=data_test["OriginalTweet"].apply(lambda x:remove_stopwords(x))

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct
data_train["OriginalTweet"]=data_train["OriginalTweet"].apply(lambda x:punct_remove(x))
data_test["OriginalTweet"]=data_test["OriginalTweet"].apply(lambda x:punct_remove(x))

print(data_train.shape)


(41157, 6)


In [165]:
X_train = data_train["OriginalTweet"]
y_train = data_train["Sentiment"]
count_vec = CountVectorizer()
X_train_counts = count_vec.fit_transform(X_train)
print(X_train_counts)

  (0, 32427)	1
  (0, 39073)	1
  (0, 8714)	1
  (1, 755)	1
  (1, 51067)	1
  (1, 34935)	2
  (1, 18071)	1
  (1, 17562)	1
  (1, 39118)	2
  (1, 36122)	2
  (1, 11917)	1
  (1, 10387)	1
  (1, 29926)	1
  (1, 45517)	1
  (1, 16647)	1
  (1, 8411)	1
  (1, 21740)	1
  (1, 46274)	1
  (1, 36894)	1
  (1, 46838)	1
  (1, 375)	1
  (1, 39963)	1
  (1, 601)	1
  (1, 50437)	1
  (1, 42720)	1
  :	:
  (41156, 11464)	1
  (41156, 35755)	1
  (41156, 1719)	1
  (41156, 43785)	1
  (41156, 21403)	1
  (41156, 40508)	1
  (41156, 47271)	1
  (41156, 40481)	1
  (41156, 31492)	1
  (41156, 56885)	1
  (41156, 11898)	1
  (41156, 42272)	1
  (41156, 43959)	1
  (41156, 1648)	1
  (41156, 57945)	1
  (41156, 2219)	1
  (41156, 55242)	1
  (41156, 56157)	1
  (41156, 1378)	1
  (41156, 23689)	1
  (41156, 23059)	1
  (41156, 51186)	1
  (41156, 35177)	1
  (41156, 43779)	1
  (41156, 57191)	1


In [166]:
print(X_train[1])
print()
print(X_train_counts[1])

advice talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist gp set online shopping accounts poss adequate supplies regular meds order

  (0, 755)	1
  (0, 51067)	1
  (0, 34935)	2
  (0, 18071)	1
  (0, 17562)	1
  (0, 39118)	2
  (0, 36122)	2
  (0, 11917)	1
  (0, 10387)	1
  (0, 29926)	1
  (0, 45517)	1
  (0, 16647)	1
  (0, 8411)	1
  (0, 21740)	1
  (0, 46274)	1
  (0, 36894)	1
  (0, 46838)	1
  (0, 375)	1
  (0, 39963)	1
  (0, 601)	1
  (0, 50437)	1
  (0, 42720)	1
  (0, 32255)	1
  (0, 37172)	1


In [167]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf[1])

  (0, 51067)	0.16266273760720704
  (0, 50437)	0.12155836858353725
  (0, 46838)	0.08606697659992589
  (0, 46274)	0.14662095272400702
  (0, 45517)	0.17143398061045428
  (0, 42720)	0.16675598189435184
  (0, 39963)	0.26883710743892564
  (0, 39118)	0.3266548371872603
  (0, 37172)	0.12816491978007186
  (0, 36894)	0.08812285189685298
  (0, 36122)	0.3447782362318782
  (0, 34935)	0.39033391979659116
  (0, 32255)	0.2030752477133528
  (0, 29926)	0.14168611930371636
  (0, 21740)	0.2162092436597497
  (0, 18071)	0.13129939899926027
  (0, 17562)	0.19775795402750743
  (0, 16647)	0.20877564332564164
  (0, 11917)	0.1677437331183683
  (0, 10387)	0.1444954305240548
  (0, 8411)	0.20483254507471538
  (0, 755)	0.14885403759536747
  (0, 601)	0.20143519347940886
  (0, 375)	0.191201102090543


In [168]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [169]:
X_test = data_test["OriginalTweet"]
y_test = data_test["Sentiment"]

X_test_counts = count_vec.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
y_pred = model.predict(X_test_tfidf)

print(accuracy_score(y_test, y_pred))

0.6324381253291206


In [170]:
text_model = Pipeline([('count_vec', CountVectorizer()), 
                     ('tfidf_transformer', TfidfTransformer()),
                     ('text_model', SGDClassifier(tol=None, loss='log'))])

text_model.fit(X_train, y_train)
y_pred = text_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.721958925750395


In [172]:
y_pred = pd.DataFrame(y_pred)

In [173]:
y_pred.value_counts()

Positive    1826
Negative    1703
Neutral      269
dtype: int64

In [174]:
y_test.value_counts()

Negative    1633
Positive    1546
Neutral      619
Name: Sentiment, dtype: int64